# 0. Preparation

In [1]:
import sys
sys.path.append('../dataset/')
sys.path.append('../network/')
sys.path.append('../model/')

import os
import glob
import time
import torch
import argparse
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision.transforms as transforms

from PIL import Image
from pathlib import Path
from main_loading import *
from main_network import *
from main_model_one_class import *
from main_model_rec import *

In [5]:
dataset_dict = {}
kmnist_loader_name = 'kmnist_eval'
kmnist_root = '/net/leksai/data/KMNIST'
kmnist_name_list = ['お-o', 'き-ki', 'す-su', 'つ-tsu', 'な-na', 'は-ha', 'ま-ma', 'や-ya', 'れ-re', 'を-wo']

for i, x in enumerate(kmnist_name_list):
    dataset_dict[x] = load_dataset(loader_name=kmnist_loader_name,
                                   root=kmnist_root,
                                   label_eval=(i,))

In [6]:
fmnist_loader_name = 'fmnist_eval'
fmnist_root = '/net/leksai/data/FashionMNIST'
fmnist_name_list = ['tshirt', 'trouser','pullover', 'dress', 'coat','sandal','shirt','sneaker', 'bag', 'boot']

for i, x in enumerate(fmnist_name_list):
    dataset_dict[x] = load_dataset(loader_name=fmnist_loader_name,
                                   root=fmnist_root,
                                   label_eval=(i,))

# 1. Metric 1: $L^{2}$-Distance for Mean of Raw Images

## 1.0. Calculating Mean Vector For Each Dataset

In [8]:
name_list = kmnist_name_list + fmnist_name_list

In [9]:
batch_size_loading = 7000
num_workers = 0
mean_dict = {}
untransform = transforms.ToPILImage()

In [10]:
for key in name_list:
    dataset = dataset_dict[key]
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        break
    
    X_mean = torch.mean(X, axis=0)
    mean_dict[key] = X_mean
    print(key)

お-o
き-ki
す-su
つ-tsu
な-na
は-ha
ま-ma
や-ya
れ-re
を-wo
tshirt
trouser
pullover
dress
coat
sandal
shirt
sneaker
bag
boot


## 1.1. Obtaining $L^2$ Distance

### 1.1.1. Get `l2_distance_df`

In [11]:
l2_distance_matrix = np.zeros((20, 20))
l2_binary_dict = dict()
criterion = nn.MSELoss(reduction='none')

In [12]:
for i, key_i in enumerate(name_list):
    for j, key_j in enumerate(name_list):
        x_i, x_j = mean_dict[key_i], mean_dict[key_j]
        dist = criterion(x_i, x_j)
        dist_mean = round(torch.mean(dist).cpu().data.numpy().item(), 3)
        l2_distance_matrix[i][j] = dist_mean

In [14]:
l2_distance_df = pd.DataFrame(l2_distance_matrix)
l2_distance_df.index = name_list
l2_distance_df.columns = name_list
l2_distance_df

,お-o,き-ki,す-su,つ-tsu,な-na,は-ha,ま-ma,や-ya,れ-re,を-wo,tshirt,trouser,pullover,dress,coat,sandal,shirt,sneaker,bag,boot
お-o,0.000,0.060,0.053,0.037,0.042,0.025,0.069,0.033,0.044,0.034,0.106,0.131,0.095,0.116,0.118,0.040,0.077,0.060,0.058,0.105
き-ki,0.060,0.000,0.008,0.040,0.019,0.036,0.014,0.034,0.009,0.018,0.058,0.046,0.083,0.042,0.092,0.030,0.050,0.058,0.077,0.095
す-su,0.053,0.008,0.000,0.039,0.019,0.030,0.012,0.026,0.009,0.014,0.056,0.042,0.083,0.037,0.093,0.032,0.050,0.058,0.082,0.106
つ-tsu,0.037,0.040,0.039,0.000,0.037,0.017,0.048,0.029,0.036,0.037,0.100,0.103,0.101,0.107,0.127,0.034,0.077,0.068,0.083,0.103
な-na,0.042,0.019,0.019,0.037,0.000,0.042,0.023,0.025,0.022,0.018,0.039,0.057,0.051,0.041,0.062,0.044,0.028,0.072,0.051,0.091
は-ha,0.025,0.036,0.030,0.017,0.042,0.000,0.043,0.024,0.026,0.032,0.106,0.099,0.108,0.099,0.131,0.021,0.081,0.048,0.081,0.104
ま-ma,0.069,0.014,0.012,0.048,0.023,0.043,0.000,0.030,0.019,0.025,0.051,0.036,0.078,0.026,0.085,0.045,0.046,0.071,0.079,0.094
や-ya,0.033,0.034,0.026,0.029,0.025,0.024,0.030,0.000,0.029,0.025,0.066,0.068,0.071,0.061,0.084,0.036,0.049,0.043,0.053,0.087
れ-re,0.044,0.009,0.009,0.036,0.022,0.026,0.019,0.029,0.000,0.017,0.061,0.054,0.074,0.048,0.084,0.029,0.045,0.055,0.065,0.098
を-wo,0.034,0.018,0.014,0.037,0.018,0.032,0.025,0.025,0.017,0.000,0.056,0.072,0.074,0.056,0.092,0.030,0.047,0.051,0.060,0.087


In [23]:
l2_distance_df['shirt'].sort_values()

shirt       0.000
pullover    0.009
coat        0.012
tshirt      0.017
な-na        0.028
dress       0.035
bag         0.036
れ-re        0.045
ま-ma        0.046
を-wo        0.047
や-ya        0.049
き-ki        0.050
す-su        0.050
trouser     0.062
お-o         0.077
つ-tsu       0.077
は-ha        0.081
boot        0.085
sandal      0.088
sneaker     0.103
Name: shirt, dtype: float64

In [22]:
l2_distance_df['は-ha'].sort_values()

は-ha        0.000
つ-tsu       0.017
sandal      0.021
や-ya        0.024
お-o         0.025
れ-re        0.026
す-su        0.030
を-wo        0.032
き-ki        0.036
な-na        0.042
ま-ma        0.043
sneaker     0.048
bag         0.081
shirt       0.081
trouser     0.099
dress       0.099
boot        0.104
tshirt      0.106
pullover    0.108
coat        0.131
Name: は-ha, dtype: float64

In [20]:
l2_distance_df['お-o'].sort_values()

お-o         0.000
は-ha        0.025
や-ya        0.033
を-wo        0.034
つ-tsu       0.037
sandal      0.040
な-na        0.042
れ-re        0.044
す-su        0.053
bag         0.058
き-ki        0.060
sneaker     0.060
ま-ma        0.069
shirt       0.077
pullover    0.095
boot        0.105
tshirt      0.106
dress       0.116
coat        0.118
trouser     0.131
Name: お-o, dtype: float64

In [9]:
l2_distance_df['お-o'].sort_values()

お-o      0.000
は-ha     0.025
や-ya     0.033
を-wo     0.034
つ-tsu    0.037
な-na     0.042
れ-re     0.044
す-su     0.053
き-ki     0.060
ま-ma     0.069
Name: お-o, dtype: float64

In [33]:
l2_distance_df['き-ki'].sort_values()

き-ki     0.000
す-su     0.008
れ-re     0.009
ま-ma     0.014
を-wo     0.018
な-na     0.019
や-ya     0.034
は-ha     0.036
つ-tsu    0.040
お-o      0.060
Name: き-ki, dtype: float64

In [35]:
l2_distance_df['す-su'].sort_values()

す-su     0.000
き-ki     0.008
れ-re     0.009
ま-ma     0.012
を-wo     0.014
な-na     0.019
や-ya     0.026
は-ha     0.030
つ-tsu    0.039
お-o      0.053
Name: す-su, dtype: float64

In [21]:
l2_distance_df['ま-ma'].sort_values()

ま-ma        0.000
す-su        0.012
き-ki        0.014
れ-re        0.019
な-na        0.023
を-wo        0.025
dress       0.026
や-ya        0.030
trouser     0.036
は-ha        0.043
sandal      0.045
shirt       0.046
つ-tsu       0.048
tshirt      0.051
お-o         0.069
sneaker     0.071
pullover    0.078
bag         0.079
coat        0.085
boot        0.094
Name: ま-ma, dtype: float64

In [36]:
l2_distance_df['や-ya'].sort_values()

や-ya     0.000
は-ha     0.024
な-na     0.025
を-wo     0.025
す-su     0.026
つ-tsu    0.029
れ-re     0.029
ま-ma     0.030
お-o      0.033
き-ki     0.034
Name: や-ya, dtype: float64

In [47]:
l2_distance_df['を-wo'].sort_values()

を-wo     0.000
す-su     0.014
れ-re     0.017
き-ki     0.018
な-na     0.018
ま-ma     0.025
や-ya     0.025
は-ha     0.032
お-o      0.034
つ-tsu    0.037
Name: を-wo, dtype: float64

> The **maximum** distance is in `<boot, trouser>`, which is **0.147**.   
> The **minimum** distance is in `<coat, pullover>`, which is **0.008**.

> The ranking for `coat` is: 
- **`pullover`** < `shirt` < `tshirt` < `bag` < `dress` < `trouser` < `boot` < `sandal` < **`sneaker`**  

> The ranking for `shirt` is: 
- **`pullover`** < `coat` < `tshirt` < `dress` < `bag` < `trouser` < `boot` < `sandal` < **`sneaker`**  

> The ranking for `pullover` is: 
- **`coat`** < `shirt` < `tshirt` < `bag` < `dress` < `trouser` < `boot` < `sandal` < **`sneaker`**  

> The ranking for `tshirt` is: 
- **`shirt`** < `dress` < `pullover` < `coat` < `trouser` < `bag` < `sandal` < `boot` < **`sneaker`**  

### 1.1.2. Get `l2_binary_df`

In [38]:
for i, key in enumerate(keys):
    distances = l2_distance_df[key].values.copy()
    max_ = keys[np.argmax(distances)]
    distances[i] = np.inf
    min_ = keys[np.argmin(distances)]
    l2_binary_dict[key] = dict()
    l2_binary_dict[key]['min'] = min_
    l2_binary_dict[key]['max'] = max_

In [39]:
l2_binary_df = pd.DataFrame(l2_binary_dict).T
l2_binary_df

,max,min
お-o,ま-ma,は-ha
き-ki,お-o,す-su
す-su,お-o,き-ki
つ-tsu,ま-ma,は-ha
な-na,お-o,を-wo
は-ha,ま-ma,つ-tsu
ま-ma,お-o,す-su
や-ya,き-ki,は-ha
れ-re,お-o,き-ki
を-wo,つ-tsu,す-su


# 2. Metric 2: $L^{1}$-Distance for Mean of Raw Images

## 2.1. Obtaining $L^1$ Distance

### 2.1.1. Get `l1_distance_df`

In [40]:
l1_distance_matrix = np.zeros((10, 10))
l1_binary_dict = dict()

In [41]:
for i, key_i in enumerate(keys):
    for j, key_j in enumerate(keys):
        x_i, x_j = mean_dict[key_i], mean_dict[key_j]
        dist = torch.abs(x_i - x_j)
        dist_mean = round(torch.mean(dist).cpu().data.numpy().item(), 3)
        l1_distance_matrix[i][j] = dist_mean

In [42]:
l1_distance_df = pd.DataFrame(l1_distance_matrix)
l1_distance_df.index = keys
l1_distance_df.columns = keys
l1_distance_df

,お-o,き-ki,す-su,つ-tsu,な-na,は-ha,ま-ma,や-ya,れ-re,を-wo
お-o,0.000,0.186,0.173,0.144,0.157,0.115,0.199,0.132,0.165,0.136
き-ki,0.186,0.000,0.061,0.156,0.099,0.137,0.077,0.135,0.066,0.101
す-su,0.173,0.061,0.000,0.150,0.104,0.124,0.068,0.112,0.067,0.088
つ-tsu,0.144,0.156,0.150,0.000,0.149,0.099,0.165,0.120,0.147,0.146
な-na,0.157,0.099,0.104,0.149,0.000,0.151,0.110,0.122,0.113,0.098
は-ha,0.115,0.137,0.124,0.099,0.151,0.000,0.149,0.111,0.121,0.136
ま-ma,0.199,0.077,0.068,0.165,0.110,0.149,0.000,0.126,0.094,0.117
や-ya,0.132,0.135,0.112,0.120,0.122,0.111,0.126,0.000,0.127,0.122
れ-re,0.165,0.066,0.067,0.147,0.113,0.121,0.094,0.127,0.000,0.101
を-wo,0.136,0.101,0.088,0.146,0.098,0.136,0.117,0.122,0.101,0.000


> The **maximum** distance is in `<coat, sandal>`, which is **0.297**.   
> The **minimum** distance is in `<coat, pullover>`, which is **0.066**.

> The ranking for `coat` is: 
- **`pullover`** < `shirt` < `tshirt` < `bag` < `dress` < `trouser` < `boot` < `sneaker` < **`sandal`**

> The ranking for `shirt` is: 
- **`pullover`** < `coat` < `tshirt` < `dress` < `bag` < `trouser` < `boot` < `sandal` < **`sneaker`**  

> The ranking for `pullover` is: 
- **`coat`** < `shirt` < `tshirt` < `bag` < `dress` < `trouser` < `boot` < `sneaker` < **`sandal`**  

> The ranking for `tshirt` is: 
- **`shirt`** < `dress` < `pullover` < `coat` < `trouser` < `bag` < `boot` < `sandal` < **`sneaker`**  

### 2.1.2. Get `l1_binary_df`

In [43]:
for i, key in enumerate(keys):
    distances = l1_distance_df[key].values.copy()
    max_ = keys[np.argmax(distances)]
    distances[i] = np.inf
    min_ = keys[np.argmin(distances)]
    l1_binary_dict[key] = dict()
    l1_binary_dict[key]['min'] = min_
    l1_binary_dict[key]['max'] = max_

In [44]:
l1_binary_df = pd.DataFrame(l1_binary_dict).T

In [46]:
l1_binary_df

,max,min
お-o,ま-ma,は-ha
き-ki,お-o,す-su
す-su,お-o,き-ki
つ-tsu,ま-ma,は-ha
な-na,お-o,を-wo
は-ha,な-na,つ-tsu
ま-ma,お-o,す-su
や-ya,き-ki,は-ha
れ-re,お-o,き-ki
を-wo,つ-tsu,す-su


In [45]:
(l2_binary_df == l1_binary_df).sum()

max     9
min    10
dtype: int64

> The results are similar to that of $L^2$-distance's.

# 3. Metric 3: $L^{2}$-Distance for Mean of Representations in Reconstruction Model (Unsupervised)

> **Step 1**: We will use coat-train to train an unsupervised reconstruction model.  
> **Step 2**: We will use coat-test to calculate the mean of representations (outputs).   
> **Step 3**: We will use other other datasets to calculate the mean of representations.  
> **Step 4**: We compare the distances between these means.   

## Shirt

In [44]:
label_eval=(6,)
base = 'shirt'

In [45]:
rec_distance_dict = {base:{base: 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_rec'
model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[{}]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'.format(label_eval[0])

net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

data_base_test = load_dataset(loader_name='fmnist_eval',
                              label_eval=label_eval,
                              test_eval=True)

base_loader_test = data_base_test.loaders(batch_size=batch_size_loading,
                                          num_workers=num_workers)


for data in base_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)
    
X_pred_mean_base = torch.mean(X_pred, axis=0)


for key, dataset in sorted(dataset_dict.items()):
    if key == base:
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_base, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    rec_distance_dict[base][key] = dist_mean

In [46]:
rec_distance_df_shirt = pd.DataFrame(rec_distance_dict)
rec_distance_df_shirt.sort_values(base, inplace=True)
rec_distance_df_shirt

,shirt
shirt,0.000000
pullover,0.000620
coat,0.000726
tshirt,0.001979
bag,0.002640
dress,0.002952
boot,0.003957
trouser,0.004746
sandal,0.004897
sneaker,0.005144


In [47]:
rec_ranking_shirt = rec_distance_df_shirt.index
rec_ranking_shirt

Index(['shirt', 'pullover', 'coat', 'tshirt', 'bag', 'dress', 'boot',
       'trouser', 'sandal', 'sneaker'],
      dtype='object')

> The ranking for `shirt` is: 
- **`pullover`** < `coat` < `tshirt` < `bag` < `dress` < `boot` < `trouser` < `sandal` < **`sneaker`**  

## Pullover

In [48]:
label_eval=(2,)
base = 'pullover'

In [49]:
rec_distance_dict = {base:{base: 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_rec'
model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[{}]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'.format(label_eval[0])

net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

data_base_test = load_dataset(loader_name='fmnist_eval',
                              label_eval=label_eval,
                              test_eval=True)

base_loader_test = data_base_test.loaders(batch_size=batch_size_loading,
                                          num_workers=num_workers)


for data in base_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)
    
X_pred_mean_base = torch.mean(X_pred, axis=0)


for key, dataset in sorted(dataset_dict.items()):
    if key == base:
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_base, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    rec_distance_dict[base][key] = dist_mean

In [50]:
rec_distance_df_pullover = pd.DataFrame(rec_distance_dict)
rec_distance_df_pullover.sort_values(base, inplace=True)
rec_distance_df_pullover

,pullover
pullover,0.000000
coat,0.000449
shirt,0.000606
tshirt,0.000968
dress,0.001964
trouser,0.002877
bag,0.003092
boot,0.005315
sneaker,0.006766
sandal,0.008275


In [51]:
rec_ranking_pullover = rec_distance_df_pullover.index
rec_ranking_pullover

Index(['pullover', 'coat', 'shirt', 'tshirt', 'dress', 'trouser', 'bag',
       'boot', 'sneaker', 'sandal'],
      dtype='object')

> The ranking for `pullover` is: 
- **`coat`** < `shirt` < `tshirt` < `dress` < `trouser` < `bag` < `boot` < `sneaker` < **`sandal`**  

## tshirt

In [52]:
label_eval=(0,)
base = 'tshirt'

In [53]:
rec_distance_dict = {base:{base: 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_rec'
model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[{}]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'.format(label_eval[0])

net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

data_base_test = load_dataset(loader_name='fmnist_eval',
                              label_eval=label_eval,
                              test_eval=True)

base_loader_test = data_base_test.loaders(batch_size=batch_size_loading,
                                          num_workers=num_workers)


for data in base_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)
    
X_pred_mean_base = torch.mean(X_pred, axis=0)


for key, dataset in sorted(dataset_dict.items()):
    if key == base:
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_base, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    rec_distance_dict[base][key] = dist_mean

In [54]:
rec_distance_df_tshirt = pd.DataFrame(rec_distance_dict)
rec_distance_df_tshirt.sort_values(base, inplace=True)
rec_distance_df_tshirt

,tshirt
tshirt,0.000000
pullover,0.000860
shirt,0.001122
coat,0.001289
dress,0.002390
trouser,0.004361
bag,0.005570
boot,0.008033
sneaker,0.008674
sandal,0.012165


In [55]:
rec_ranking_tshirt = rec_distance_df_tshirt.index
rec_ranking_tshirt

Index(['tshirt', 'pullover', 'shirt', 'coat', 'dress', 'trouser', 'bag',
       'boot', 'sneaker', 'sandal'],
      dtype='object')

> The ranking for `tshirt` is: 
- **`pullover`** < `shirt` < `coat` < `dress` < `trouser` < `bag` < `boot` < `sneaker` < **`sandal`**  

## 3.1. Get Mean

In [33]:
label_eval=(4,)
base = 'coat'

In [31]:
rec_distance_dict = {base:{base: 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_rec'
model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[{}]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'.format(label_eval[0])

In [34]:
net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

<All keys matched successfully>

In [36]:
data_base_test = load_dataset(loader_name='fmnist_eval',
                              label_eval=label_eval,
                              test_eval=True)

base_loader_test = data_base_test.loaders(batch_size=batch_size_loading,
                                     num_workers=num_workers)

In [38]:
for data in base_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)

In [39]:
X_pred_mean_base = torch.mean(X_pred, axis=0)

In [40]:
display(untransform(X_pred_mean_base))

## 3.2. Get Distances

In [41]:
for key, dataset in sorted(dataset_dict.items()):
    if key == base:
        continue
    print(key)
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_base, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    rec_distance_dict[base][key] = dist_mean

bag
boot
dress
pullover
sandal
shirt
sneaker
trouser
tshirt


In [42]:
rec_distance_df = pd.DataFrame(rec_distance_dict)
rec_distance_df.sort_values(base, inplace=True)
rec_distance_df

,coat
coat,0.000000
pullover,0.000900
shirt,0.001395
bag,0.002755
tshirt,0.003577
boot,0.006922
dress,0.007182
sneaker,0.009563
sandal,0.010589
trouser,0.011027


In [43]:
rec_ranking = rec_distance_df.index
rec_ranking

Index(['coat', 'pullover', 'shirt', 'bag', 'tshirt', 'boot', 'dress',
       'sneaker', 'sandal', 'trouser'],
      dtype='object')

> The **maximum** distance is in `<coat, sandal>`, which is **0.011**.   
> The **minimum** distance is in `<coat, pullover>`, which is **0.001**.  

> The ranking for `coat` is: 
- **`pullover`** < `shirt` < `bag` < `tshirt` < `boot` < `dress` < `sneaker` < `sandal` < **`trouser`**  

> The ranking for `shirt` is: 
- **`pullover`** < `coat` < `tshirt` < `bag` < `dress` < `boot` < `trouser` < `sandal` < **`sneaker`**   

> The ranking for `pullover` is: 
- **`coat`** < `shirt` < `tshirt` < `dress` < `trouser` < `bag` < `boot` < `sneaker` < **`sandal`**  

> The ranking for `tshirt` is: 
- **`pullover`** < `shirt` < `coat` < `dress` < `trouser` < `bag` < `boot` < `sneaker` < **`sandal`**  

# 4. Metric 4: $L^{2}$-Distance for Mean of Representations in One-Class Model (Unsupervised, w/o Pretraining)

> **Step 1**: We will use coat-train to train an unsupervised one-class model.  
> **Step 2**: We will use coat-test to calculate the mean of representations (outputs).   
> **Step 3**: We will use other other datasets to calculate the mean of representations.  
> **Step 4**: We compare the distances between these means.   

## 4.1. Get Mean

In [361]:
oc_distance_dict = {'coat':{'coat': 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_one_class'
model_path = '/net/leksai/nips/model/one_class/fmnist/one_class_unsupervised_[4]_[]_[0]_[0.0]/net_fmnist_LeNet_one_class_eta_100_epochs_150_batch_128/model.tar'

In [362]:
net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

<All keys matched successfully>

In [363]:
for data in coat_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)

In [364]:
X_pred_mean_coat = torch.mean(X_pred, axis=0)

## 4.2. Get Distances

In [365]:
for key, dataset in sorted(dataset_dict.items()):
    if key == 'coat':
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_coat, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    oc_distance_dict['coat'][key] = dist_mean

In [378]:
oc_distance_df = pd.DataFrame(oc_distance_dict)
oc_distance_df.sort_values('coat', inplace=True)
oc_distance_df

,coat
coat,0.000000
sneaker,0.000007
bag,0.000008
tshirt,0.000013
dress,0.000014
boot,0.000018
sandal,0.000032
pullover,0.000044
shirt,0.000069
trouser,0.000212


In [381]:
oc_ranking = oc_distance_df.index
oc_ranking

Index(['coat', 'sneaker', 'bag', 'tshirt', 'dress', 'boot', 'sandal',
       'pullover', 'shirt', 'trouser'],
      dtype='object')

> The **maximum** distance is in `<coat, trouser>`, which is **0.000212**.   
> The **minimum** distance is in `<coat, sneaker>`, which is **0.000007**.  

> The ranking for `coat` is: 
- **`sneaker`** < `bag` < `tshirt` < `dress` < `boot` < `sandal` < `pullover` < `shirt` < **`trouser`**

> The ranking is quite different to raw images, and the magnitudes are smaller.

# 5. Metric 5: $L^{2}$-Distance for Mean of Representations in One-Class Model (Unsupervised, w/ Pretraining)

> **Step 1**: We will use coat-train to train an unsupervised one-class model.  
> **Step 2**: We will use coat-test to calculate the mean of representations (outputs).   
> **Step 3**: We will use other other datasets to calculate the mean of representations.  
> **Step 4**: We compare the distances between these means.   

## 5.1. Get Mean

In [353]:
oc_pt_distance_dict = {'coat':{'coat': 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_one_class'
model_path = '/net/leksai/nips/model/one_class/fmnist/one_class_unsupervised_[4]_[]_[1]_[0.0]/net_fmnist_LeNet_one_class_eta_100_epochs_150_batch_128/model.tar'

In [354]:
net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

<All keys matched successfully>

In [355]:
for data in coat_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)

In [356]:
X_pred_mean_coat = torch.mean(X_pred, axis=0)

## 5.2. Get Distances

In [411]:
for key, dataset in sorted(dataset_dict.items()):
    if key == 'coat':
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_coat, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    oc_pt_distance_dict['coat'][key] = dist_mean

In [371]:
oc_pt_distance_df = pd.DataFrame(oc_pt_distance_dict)
oc_pt_distance_df.sort_values('coat', inplace=True)

In [372]:
oc_pt_distance_df

,coat
coat,0.000000
bag,0.000007
sneaker,0.000007
dress,0.000009
tshirt,0.000012
boot,0.000013
sandal,0.000028
pullover,0.000049
shirt,0.000086
trouser,0.000220


In [380]:
oc_pt_ranking = oc_pt_distance_df.index
oc_pt_ranking

Index(['coat', 'bag', 'sneaker', 'dress', 'tshirt', 'boot', 'sandal',
       'pullover', 'shirt', 'trouser'],
      dtype='object')

> The **maximum** distance is in `<coat, trouser>`, which is **0.000200**.   
> The **minimum** distance is in `<coat, bag>` and `<coat, sneaker>`, which are respectively **6.6e-6** and **7.7e-6**.  

> The ranking for `coat` is: 
- **`bag`** < `sneaker` < `dress` < `tshirt` < `boot` < `sandal` < `pullover` < `shirt` < **`trouser`** 

> The ranking is quite different to raw images, and the magnitudes are smaller.